In [47]:
!pip install -q gdown
!pip install -q airportsdata
!pip install -q dash


In [48]:
import sys
print(sys.executable)

/Users/jumpeikasai/Documents/2025 Spring/.venv/bin/python


In [49]:
import pandas as pd
import duckdb
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import gdown
import plotly.graph_objects as go
import plotly.express as px
from airportsdata import load


In [50]:
url = 'https://drive.google.com/uc?id=1SDQdskYjhUNWKRlKwSa189BenJsgHl8E'
gdown.download(url, 'airports.csv', quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1SDQdskYjhUNWKRlKwSa189BenJsgHl8E
To: /Users/jumpeikasai/Documents/2025 Spring/DS260/FinalProject/airports.csv
100%|██████████| 63.0M/63.0M [00:01<00:00, 34.3MB/s]


'airports.csv'

In [51]:
mainDF = pd.read_csv('airports.csv', quoting = 1)
mainDF.drop(columns = ['Geocoded_City1', 'Geocoded_City2'], inplace = True)
mainDF

/var/folders/_r/40h75qsj5jl6l256vlbqm8lm0000gn/T/ipykernel_16560/479330524.py:1: DtypeWarning:

Columns (20,21) have mixed types. Specify dtype option on import or set low_memory=False.



,tbl,Year,quarter,citymarketid_1,citymarketid_2,city1,city2,airportid_1,airportid_2,airport_1,...,nsmiles,passengers,fare,carrier_lg,large_ms,fare_lg,carrier_low,lf_ms,fare_low,tbl1apk
0,Table1a,2021,3,30135,33195,"Allentown/Bethlehem/Easton, PA","Tampa, FL (Metropolitan Area)",10135,14112,ABE,...,970,180,81.43,G4,1.0000,81.43,G4,1.0000,81.43,202131013514112ABEPIE
1,Table1a,2021,3,30135,33195,"Allentown/Bethlehem/Easton, PA","Tampa, FL (Metropolitan Area)",10135,15304,ABE,...,970,19,208.93,DL,0.4659,219.98,UA,0.1193,154.11,202131013515304ABETPA
2,Table1a,2021,3,30140,30194,"Albuquerque, NM","Dallas/Fort Worth, TX",10140,11259,ABQ,...,580,204,184.56,WN,0.9968,184.44,WN,0.9968,184.44,202131014011259ABQDAL
3,Table1a,2021,3,30140,30194,"Albuquerque, NM","Dallas/Fort Worth, TX",10140,11298,ABQ,...,580,264,182.64,AA,0.9774,183.09,AA,0.9774,183.09,202131014011298ABQDFW
4,Table1a,2021,3,30140,30466,"Albuquerque, NM","Phoenix, AZ",10140,14107,ABQ,...,328,398,177.11,WN,0.6061,184.49,AA,0.3939,165.77,202131014014107ABQPHX
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245950,Table1a,2024,1,35412,31703,"Knoxville, TN","New York City, NY (Metropolitan Area)",15412,12953,TYS,...,665,207,278.70,DL,0.7503,287.44,AA,0.2359,248.46,202411541212953TYSLGA
245951,Table1a,2024,1,35412,32467,"Knoxville, TN","Miami, FL (Metropolitan Area)",15412,11697,TYS,...,724,277,148.69,G4,0.8255,114.45,G4,0.8255,114.45,202411541211697TYSFLL
245952,Table1a,2024,1,35412,32467,"Knoxville, TN","Miami, FL (Metropolitan Area)",15412,13303,TYS,...,724,70,330.19,AA,0.8057,321.92,AA,0.8057,321.92,202411541213303TYSMIA
245953,Table1a,2024,1,35412,33195,"Knoxville, TN","Tampa, FL (Metropolitan Area)",15412,14112,TYS,...,550,178,95.65,G4,1.0000,95.65,G4,1.0000,95.65,202411541214112TYSPIE


In [52]:
def fixAirports(colName):
  for i in range(len(mainDF)):
    val = mainDF.loc[i, colName]
    if(val == "WHR"):
      mainDF.loc[i, colName] = "EGE"
    if(val == "BHC"):
      mainDF.loc[i, colName] = "IFP"
    if (val == "DTT"):
      mainDF.loc[i, colName] = "DTW"
fixAirports('airport_1')
fixAirports('airport_2')

In [53]:
airports = load("IATA")

Manually Adding Missing Airports

In [54]:
airports.update({"AIY": {"lon": -74.4561005, "lat": 39.3600006, 'name': 'Bader Field'},
                 "WAS": {"lat": 38.889805, "lon": -77.009056, 'name': 'Washington Airports'},
                 "JRB": {"lat": 40.700711, "lon": -74.008573, 'name': 'Downtown Manhattan Heliport'},
                 "TSS": {"lat": 40.742372, "lon": -73.972074, 'name':'East 34th St Heliport'},
                 "MPB": {"lat": 25.778299, "lon": -80.170303, "name": 'Miami Seaplane Base'},
                 "CGX": {"lat": 41.8596, "lon": -87.6084, "name": 'Meigs Field'},
                 "NYC": {"lat": 40.7128, "lon": -74.0060, "name": "New York Airports"},
                 "CHI": {"lat": 41.8781, "lon": -87.6298, "name": "Chicago Airports"}
                 })

In [55]:
lat1 = [airports[ap]["lat"] for ap in mainDF['airport_1']]
lat2 = [airports[ap]["lat"] for ap in mainDF['airport_2']]
lon1 = [airports[ap]["lon"] for ap in mainDF['airport_1']]
lon2 = [airports[ap]["lon"] for ap in mainDF['airport_2']]



In [56]:
mainDF.insert(11, "lat1", lat1 )
mainDF.insert(12, "lat2", lat2 )
mainDF.insert(13, "lon1", lon1 )
mainDF.insert(14, "lon2", lon2 )

In [57]:
mainDF['airport_1'][65945]

'ORD'

In [58]:
def filterYear(df, year, endYear = None):
  if endYear != None:
    qString = "SELECT * FROM df WHERE Year >= " + str(year) + " AND Year <= " + str(endYear)
  else:
    qString = "SELECT * FROM df WHERE Year = " + str(year)
  return duckdb.query(qString).df()

filterYear(mainDF, 2015, 2018)


,tbl,Year,quarter,citymarketid_1,citymarketid_2,city1,city2,airportid_1,airportid_2,airport_1,...,nsmiles,passengers,fare,carrier_lg,large_ms,fare_lg,carrier_low,lf_ms,fare_low,tbl1apk
0,Table 1a,2015,4,30977,34986,"Chicago, IL","Sarasota/Bradenton, FL",13232,14986,MDW,...,1050,12,234.53,DL,1.0000,234.53,DL,1.0000,234.53,201541323214986MDWSRQ
1,Table 1a,2015,3,31453,33244,"Houston, TX","Memphis, TN",12266,13244,IAH,...,484,160,233.68,UA,0.8700,232.68,UA,0.8700,232.68,201531226613244IAHMEM
2,Table 1a,2015,3,30599,30977,"Birmingham, AL","Chicago, IL",10599,13232,BHM,...,584,219,197.74,WN,0.9100,194.29,WN,0.9100,194.29,201531059913232BHMMDW
3,Table 1a,2015,3,30194,31295,"Dallas/Fort Worth, TX","Detroit, MI",11259,11433,DAL,...,986,159,166.76,WN,0.9700,164.31,WN,0.9700,164.31,201531125911433DALDTW
4,Table 1a,2015,3,30194,31454,"Dallas/Fort Worth, TX","Orlando, FL",11298,13204,DFW,...,985,1530,155.14,AA,0.7700,168.32,NK,0.1700,85.91,201531129813204DFWMCO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32799,Table1a,2018,4,31834,31703,"Fayetteville, AR","New York City, NY (Metropolitan Area)",15919,11618,XNA,...,1159,107,357.24,UA,0.7102,371.74,AA,0.1786,320.94,201841591911618XNAEWR
32800,Table1a,2018,4,31834,31703,"Fayetteville, AR","New York City, NY (Metropolitan Area)",15919,12197,XNA,...,1159,3,359.38,AA,0.6333,351.95,AA,0.6333,351.95,201841591912197XNAHPN
32801,Table1a,2018,4,31834,31703,"Fayetteville, AR","New York City, NY (Metropolitan Area)",15919,12478,XNA,...,1159,8,290.22,AA,0.5128,318.76,DL,0.3846,277.47,201841591912478XNAJFK
32802,Table1a,2018,4,31834,31703,"Fayetteville, AR","New York City, NY (Metropolitan Area)",15919,12953,XNA,...,1159,224,288.82,AA,0.7102,285.67,AA,0.7102,285.67,201841591912953XNALGA


In [59]:

def getNumOutRoutes(df: pd.DataFrame, byPassengers = False):
  quarterRoutes = duckdb.query(f"""SELECT * FROM df""").to_df()
  if byPassengers:
    distinctRoutes = duckdb.query(f"""SELECT airport_1 as airport, lat1 as lat, lon1 as lon, SUM(passengers) as num FROM
     quarterRoutes
     GROUP BY airport, lat1, lon1 ORDER BY num DESC""").to_df()
  else:
    distinctRoutes = duckdb.query(f"""SELECT airport_1 as airport, lat1 as lat, lon1 as lon, COUNT(DISTINCT airport_2) as num FROM
    quarterRoutes
    GROUP BY airport, lat1, lon1 ORDER BY num DESC""").to_df()
  return distinctRoutes



def getNumInRoutes(df: pd.DataFrame, byPassengers = False):
  quarterRoutes = duckdb.query(f"""SELECT * FROM df""").to_df()
  if byPassengers:
    distinctRoutes = duckdb.query(f"""SELECT airport_2 as airport, lat2 as lat, lon2 as lon, SUM(passengers) as num FROM
     quarterRoutes
     GROUP BY airport, lat2, lon2 ORDER BY num DESC""").to_df()
  else:
    distinctRoutes = duckdb.query(f"""SELECT airport_2 as airport, lat2 as lat, lon2 as lon, COUNT(DISTINCT airport_1) as num FROM
    quarterRoutes
    GROUP BY airport, lat, lon ORDER BY num DESC""").to_df()
  return distinctRoutes

getNumInRoutes(mainDF, True)


,airport,lat,lon,num
0,SFO,37.618810,-122.375420,4765796.0
1,LGA,40.777250,-73.872610,4117933.0
2,BWI,39.175730,-76.668990,3807021.0
3,DCA,38.851440,-77.037720,3662269.0
4,TPA,27.975470,-82.533250,3337450.0
...,...,...,...,...
167,FMY,26.586610,-81.863250,0.0
168,JRB,40.700711,-74.008573,0.0
169,CGX,41.859600,-87.608400,0.0
170,HAR,40.217140,-76.851360,0.0


In [60]:
def getRoutePassengers(df: pd.DataFrame):
  newDF = duckdb.query("""SELECT airport_1, airport_2, SUM(passengers) as totalPass
  FROM  df
  GROUP BY (airport_1, airport_2)
  ORDER BY totalPass DESC""").to_df()
  return newDF


In [61]:
df2018 = filterYear(mainDF, 2018)
outFlights = getNumOutRoutes(df2018)
limits = [(0,3),(3,11),(11,21),(21,50),(50,200)]
colors = ["royalblue","crimson","lightseagreen","orange","lightgrey"]
cities = []
scale = .5

fig = go.Figure()

for i in range(len(limits)):
    lim = limits[i]
    df_sub = outFlights[lim[0]:lim[1]]
    df_sub['text'] = df_sub['airport'] + ' - ' + df_sub['num'].astype(str)
    fig.add_trace(go.Scattergeo(
        locationmode = 'USA-states',
        lon = df_sub['lon'],
        lat = df_sub['lat'],
        text = df_sub['text'],
        marker = dict(
            size = df_sub['num']/scale,
            color = colors[i],
            line_color='rgb(40,40,40)',
            line_width=0.5,
            sizemode = 'area'
        ),
        name = '{0} - {1}'.format(lim[0],lim[1])))

fig.update_layout(
        title_text = 'US Airport Flights Out<br>(Click legend to toggle traces)',
        showlegend = True,
        geo = dict(
            scope = 'usa',
            landcolor = 'rgb(232, 220, 202)',
        ), legend_title_text = 'Ranking by Number of Flights'
    )
import plotly.io as pio
pio.renderers.default = "browser"  # ← これが超重要
fig.show()



/var/folders/_r/40h75qsj5jl6l256vlbqm8lm0000gn/T/ipykernel_16560/932343728.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/_r/40h75qsj5jl6l256vlbqm8lm0000gn/T/ipykernel_16560/932343728.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/_r/40h75qsj5jl6l256vlbqm8lm0000gn/T/ipykernel_16560/932343728.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [62]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
server = app.server
app.config.suppress_callback_exceptions = True

app.layout = html.Div([
    html.H1(children = 'US Airports and Flight Routes', style = {'textAlign': 'center'}),
    html.H2(children='A visualization of airport, airline, and flight route popularity'),

    html.Div([
        "Beginning year: ",
        dcc.Input(id = 'beg_year_input', value = 1993, type = 'number', debounce = True)],
             style = {'width': '22%', 'display': 'inline-block', 'marginRight': '2%'}),

    html.Div([
        "Ending year: ",
        dcc.Input(id = 'end_year_input', value = 2024, type = 'number', debounce = True)
    ], style = {'width': '22%', 'display': 'inline-block', 'marginRight': '2%'}),

    html.Div([
        "Flight status: ",
        dcc.RadioItems(["Flights In", "Flights Out"], "Flights In", id = 'in_out')
    ], style = {'width': '22%', 'display': 'inline-block', 'marginRight': '2%'}),

    html.Div([
        "Measurement: ",
        dcc.RadioItems(["Number of Flights", "Number of Passengers"], "Number of Flights", id='count_type_button')

    ], style = {'width': '22%', 'display': 'inline-block', 'marginLeft': '2%'}),

    html.Div([
        html.Div([
            dcc.Graph(id = 'out_routes_map')
        ], style = {'width': '98%', 'display': 'inline-block', 'vertical-align': 'middle'})

       #html.Div([
        #    dcc.Graph(id = 'in_routes_map')
        #], style = {'width': '49%', 'display': 'inline-block', 'vertical-align': 'middle'})

    ])



])

@app.callback(
    Output('out_routes_map', 'figure'),
    Input('beg_year_input', 'value'),
    Input('end_year_input', 'value'),
    Input('in_out', 'value'),
    Input('count_type_button', 'value')
)
def update_out_map(beg_year = 1993, end_year = 2024, status = "Flights In", count_type = "Number of Flights"):
    if beg_year == None:
        beg_year = 1993
    if end_year == None:
        end_year = beg_year
    if end_year < beg_year:
        return go.Figure().update_layout(title_text = "Please Enter Valid Year Range")

    countTypeBool = count_type == "Number of Passengers"
    if countTypeBool:
        legendTitle = "Ranking by Number of Passengers"
        #scale = 10000

    else:
        legendTitle = "Ranking by Number of Flights"
        #scale = 0.5


    df = filterYear(mainDF, beg_year, end_year)
    if status == "Flights Out":
        flights = getNumOutRoutes(df, countTypeBool)
        title = "US Airport Flights Out"
    else:
        flights = getNumInRoutes(df, countTypeBool)
        title = "US Airport Flights In"
    last = len(flights)
    limits = [(0,3),(3,11),(11,21),(21,50),(50,last)]
    colors = ["royalblue","crimson","lightseagreen","orange","lightgrey"]
    cities = []
    scale = flights.loc[0, 'num']/300

    fig = go.Figure()

    for i in range(len(limits)):
        lim = limits[i]
        df_sub = flights[lim[0]:lim[1]]
        df_sub['name'] = [airports[ap]['name'] for ap in df_sub['airport']]
        df_sub['text'] = df_sub['name'] + ' ('+df_sub['airport']+')'+'<br>' + df_sub['num'].astype(str)
        fig.add_trace(go.Scattergeo(
            locationmode = 'USA-states',
            lon = df_sub['lon'],
            lat = df_sub['lat'],
            text = df_sub['text'],
            marker = dict(
                size = df_sub['num']/scale,
                color = colors[i],
                line_color='rgb(40,40,40)',
                line_width=0.5,
                sizemode = 'area'
            ),
            name = '{0} - {1}'.format(lim[0]+1,lim[1])))

    fig.update_layout(
            title_text = title + '<br>(Click legend to toggle traces)',
            showlegend = True,
            geo = dict(
                scope = 'usa',
                #landcolor = 'rgb(232, 220, 202)',
                landcolor = 'rgb(0, 0, 0)',
            ), legend_title_text = legendTitle
        )
    return fig




In [63]:
app.run(jupyter_mode = 'external', host = '0.0.0.0', port = 8050, debug = True, use_reloader = True)

Dash app running on http://0.0.0.0:8050/


In [64]:
mainDF['airportName1'] = [airports[ap]['name'] for ap in mainDF['airport_1']]
mainDF['airportName2'] = [airports[ap]['name'] for ap in mainDF['airport_2']]
mainDF

,tbl,Year,quarter,citymarketid_1,citymarketid_2,city1,city2,airportid_1,airportid_2,airport_1,...,fare,carrier_lg,large_ms,fare_lg,carrier_low,lf_ms,fare_low,tbl1apk,airportName1,airportName2
0,Table1a,2021,3,30135,33195,"Allentown/Bethlehem/Easton, PA","Tampa, FL (Metropolitan Area)",10135,14112,ABE,...,81.43,G4,1.0000,81.43,G4,1.0000,81.43,202131013514112ABEPIE,Lehigh Valley International Airport,St Pete-Clearwater International Airport
1,Table1a,2021,3,30135,33195,"Allentown/Bethlehem/Easton, PA","Tampa, FL (Metropolitan Area)",10135,15304,ABE,...,208.93,DL,0.4659,219.98,UA,0.1193,154.11,202131013515304ABETPA,Lehigh Valley International Airport,Tampa International Airport
2,Table1a,2021,3,30140,30194,"Albuquerque, NM","Dallas/Fort Worth, TX",10140,11259,ABQ,...,184.56,WN,0.9968,184.44,WN,0.9968,184.44,202131014011259ABQDAL,Albuquerque International Sunport Airport,Dallas Love Field
3,Table1a,2021,3,30140,30194,"Albuquerque, NM","Dallas/Fort Worth, TX",10140,11298,ABQ,...,182.64,AA,0.9774,183.09,AA,0.9774,183.09,202131014011298ABQDFW,Albuquerque International Sunport Airport,Dallas-Fort Worth International Airport
4,Table1a,2021,3,30140,30466,"Albuquerque, NM","Phoenix, AZ",10140,14107,ABQ,...,177.11,WN,0.6061,184.49,AA,0.3939,165.77,202131014014107ABQPHX,Albuquerque International Sunport Airport,Phoenix Sky Harbor International Airport
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245950,Table1a,2024,1,35412,31703,"Knoxville, TN","New York City, NY (Metropolitan Area)",15412,12953,TYS,...,278.70,DL,0.7503,287.44,AA,0.2359,248.46,202411541212953TYSLGA,Mc Ghee Tyson Airport,Laguardia Airport
245951,Table1a,2024,1,35412,32467,"Knoxville, TN","Miami, FL (Metropolitan Area)",15412,11697,TYS,...,148.69,G4,0.8255,114.45,G4,0.8255,114.45,202411541211697TYSFLL,Mc Ghee Tyson Airport,Fort Lauderdale/Hollywood International Airport
245952,Table1a,2024,1,35412,32467,"Knoxville, TN","Miami, FL (Metropolitan Area)",15412,13303,TYS,...,330.19,AA,0.8057,321.92,AA,0.8057,321.92,202411541213303TYSMIA,Mc Ghee Tyson Airport,Miami International Airport
245953,Table1a,2024,1,35412,33195,"Knoxville, TN","Tampa, FL (Metropolitan Area)",15412,14112,TYS,...,95.65,G4,1.0000,95.65,G4,1.0000,95.65,202411541214112TYSPIE,Mc Ghee Tyson Airport,St Pete-Clearwater International Airport


In [66]:
filtered_df = mainDF.dropna(subset=['nsmiles', 'fare_lg', 'large_ms'])

fig = px.scatter(
    filtered_df,
    x='nsmiles',
    y='fare_lg',
    color='large_ms',
    size='large_ms',
    hover_name='airport_1',
    hover_data={'nsmiles':':.1f','fare_lg':':.0f'},
    title='Distance vs. Avg Fare (Largest Carrier)\nColor & Size = Market Share'
)
fig.show()

